In [69]:
!pip install -q datasets

In [70]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [71]:
data = [
    ["I woke up early this morning.", "من امروز صبح زود بیدار شدم."],
    ["She is reading a very interesting book.", "او دارد یک کتاب بسیار جالب می‌خواند."],
    ["They went to the park to play football.", "آن‌ها برای بازی فوتبال به پارک رفتند."],
    ["We had dinner at a nice restaurant last night.", "دیشب در یک رستوران خوب شام خوردیم."],
    ["He doesn't like watching horror movies.", "او تماشای فیلم‌های ترسناک را دوست ندارد."],
    ["Can you help me with this math problem?", "می‌تونی تو حل این مسئله ریاضی بهم کمک کنی؟"],
    ["The weather is getting colder every day.", "هوا هر روز سردتر می‌شود."],
    ["I have never been to Paris.", "من هرگز به پاریس نرفته‌ام."],
    ["She always forgets where she puts her keys.", "او همیشه فراموش می‌کند کلیدهایش را کجا گذاشته."],
    ["We are planning a trip to the mountains.", "ما داریم یک سفر به کوهستان برنامه‌ریزی می‌کنیم."]
]

In [72]:
import pandas as pd

In [73]:
DF=pd.DataFrame(data,columns=['English','Persian'])
DF

,English,Persian
0,I woke up early this morning.,من امروز صبح زود بیدار شدم.
1,She is reading a very interesting book.,او دارد یک کتاب بسیار جالب می‌خواند.
2,They went to the park to play football.,آن‌ها برای بازی فوتبال به پارک رفتند.
3,We had dinner at a nice restaurant last night.,دیشب در یک رستوران خوب شام خوردیم.
4,He doesn't like watching horror movies.,او تماشای فیلم‌های ترسناک را دوست ندارد.
5,Can you help me with this math problem?,می‌تونی تو حل این مسئله ریاضی بهم کمک کنی؟
6,The weather is getting colder every day.,هوا هر روز سردتر می‌شود.
7,I have never been to Paris.,من هرگز به پاریس نرفته‌ام.
8,She always forgets where she puts her keys.,او همیشه فراموش می‌کند کلیدهایش را کجا گذاشته.
9,We are planning a trip to the mountains.,ما داریم یک سفر به کوهستان برنامه‌ریزی می‌کنیم.


In [66]:
sources = [s for s, _ in data]
references = [t for _, t in data]

In [21]:
icl_prompt = """
Translate English sentences to Persian:

English sentence: I woke up early this morning.
Persian sentence: من امروز صبح زود بیدار شدم.

English sentence: She is reading a very interesting book.
Persian sentence: او دارد یک کتاب بسیار جالب می‌خواند.

English sentence: They went to the park to play football.
Persian sentence: آن‌ها برای بازی فوتبال به پارک رفتند.

English sentence: {}
Persian sentence:
"""

In [20]:
sources1=sources[3:10]
references1=references[3:10]

In [ ]:
import os

In [22]:
import huggingface_hub
hf_token=os.environ["hf_token"]
huggingface_hub.login(token=hf_token)

In [23]:
!huggingface-cli whoami

⚠️  Warning: 'huggingface-cli whoami' is deprecated. Use 'hf auth whoami' instead.
user:  Z-goli


In [24]:
model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [78]:
preds_llm = []
for src in sources1:
    messages = [
    {"role": "user", "content": icl_prompt.format(src)},
    ]
    inputs = tokenizer.apply_chat_template(
	    messages,
	    add_generation_prompt=True,
	    tokenize=True,
	    return_dict=True,
	    return_tensors="pt",
    ).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=40,do_sample=False)
    pred = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:],skip_special_tokens=True)
    if "Persian sentence:" in pred:
      pred = pred.split("Persian sentence:")[-1].strip()
    else:
      pred = pred.strip()
    preds_llm.append(pred)


In [79]:
preds_llm2=[p.replace("\u200c"," ") for p in preds_llm]
preds_llm2

['ما شب گذشته در یک رستوران خوب شام خوردیم.',
 'او فیلم های ترسناک را دوست ندارد.',
 'می توانید به من در این مسئله ریاضی کمک کنید؟',
 'هوا هر روز سردتر می شود.',
 "من بارها به پاریس نرفته ام.\n\n(Man bar ha va be Paris nahe'am)\n\nHere's a breakdown of why this is the most natural translation:",
 'او همیشه کلیدهای خود را در جایی که می گذارد فراموش می کند.',
 'ما یک سفر به کوه ها برنامه ریزی می کنیم.']

In [68]:
!pip install -q googletrans==4.0.0rc1

In [5]:
from googletrans import Translator

In [27]:
gt=Translator()
preds_google = [gt.translate(s, src="en", dest="fa").text for s in sources1]
print(preds_google)

['دیشب در یک رستوران خوب شام خوردیم.', 'او تماشای فیلم های ترسناک را دوست ندارد.', 'آیا می توانید در این مشکل ریاضی به من کمک کنید؟', 'هوا هر روز سردتر می شود.', 'من هرگز به پاریس نرفته ام.', 'او همیشه فراموش می کند که کلیدهای خود را قرار می دهد.', 'ما در حال برنامه ریزی سفر به کوه هستیم.']


In [56]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [80]:
# LLM BLEU calculation
smooth = SmoothingFunction().method1
for i, (ref, pred) in enumerate(zip(references1,preds_llm)):
    referencce = [ref.split()]
    generated = pred.split()
    LLmscore = sentence_bleu(referencce, generated, smoothing_function=smooth)
    print(f"Sentence {i+1} LLM BLEU score: {LLmscore:.4f}")

Sentence 1 LLM BLEU score: 0.5874
Sentence 2 LLM BLEU score: 0.6732
Sentence 3 LLM BLEU score: 0.1275
Sentence 4 LLM BLEU score: 1.0000
Sentence 5 LLM BLEU score: 0.0440
Sentence 6 LLM BLEU score: 0.0474
Sentence 7 LLM BLEU score: 0.1865


In [81]:
# Google tranlate BLEU calculation
smooth = SmoothingFunction().method1
for i, (ref, pred) in enumerate(zip(references1,preds_google)):
    referencce = [ref.split()]
    generated = pred.split()
    LLmscore = sentence_bleu(referencce, generated, smoothing_function=smooth)
    print(f"Sentence {i+1} google BLEU score: {LLmscore:.4f}")

Sentence 1 google BLEU score: 1.0000
Sentence 2 google BLEU score: 0.4111
Sentence 3 google BLEU score: 0.0248
Sentence 4 google BLEU score: 0.5081
Sentence 5 google BLEU score: 0.5081
Sentence 6 google BLEU score: 0.0906
Sentence 7 google BLEU score: 0.0561
